In [1]:
import numpy as np
import pickle
import random

In [6]:
class BlackJackSolution:

    def __init__(self, lr=0.1, exp_rate=0.3, num_decks=6):
        
        # Create two dictionaries (one to hold Q values, one to hold win/total games)
        self.player_Q_Values = {}  # [(player_value, show_card, usable_ace)][action]: Qvalue
        self.stateActionOutcome = {} # [(player_value, show_card, usable_ace)][action]: [win,totalgames]
        
        # initialize Q values | (2-21) x (1-10) x (True, False) x (0,1,2,3) x (10,50,100) creates 4800
        # combinations for Q values
        
        # i loop for player hand total
        for i in range(2, 22):
            # j loop for dealer show card
            for j in range(1, 11):
                # k loop for if player has usable ace
                for k in [True, False]:
                    
                    # initialize our win/total game values in dictionary
                    self.stateActionOutcome[(i,j,k)] = {0:[0,0],1:[0,0],2:[0,0],3:[0,0]}
                    
                    for bet in [10,50,100]:
                        
                        # initialize a dictionary for each combination of states to hold action:Qvalue
                        self.player_Q_Values[(i, j, k, bet)] = {}
                        
                        # Loop through actions (0: Stay, 1: Hit, 2: Double Down, 3: Split)
                        for a in [0, 1, 2, 3]:
               
                            # Below, we initialize the model with some very primitive rules
            
                            # If the player has a 21, then we want a choice of stay (0) to have a 
                            # very high Q value  
                            if (i == 21) and (a == 0):
                                self.player_Q_Values[(i, j, k, bet)][a] = 99999999
                                
                            # If the player has less than 12, then staying is very dumb 
                            # so we set the Q value to a very negative number
                            elif (i < 12) and (a == 0):
                                self.player_Q_Values[(i, j, k, bet)][a] = -99999999
                                
                            # If you have two aces, always split (3)
                            elif (i == 2) and (a == 3):
                                self.player_Q_Values[(i, j, k, bet)][a] = 99999999
                            
                            # Otherwise, set the Q value to 0
                            else:
                                self.player_Q_Values[(i, j, k, bet)][a] = 0

        # Create attributes to keep track of various states, actions, etc.
        self.player_state_action = []
        self.state = (0, 0, False, 10)  # initial state
        self.actions = [1, 0, 2, 3]
        self.end = False # Used to end game
        self.lr = lr
        self.exp_rate = exp_rate
        self.num_decks = num_decks
        self.testStack = []
        
        # Action count is used to keep track of how many actions have been performed in a game
        self.actionCount = 0
        self.bet = 0
        
        # Initial thought: Have it be 3 elements (state on which split occurred, states for hand 1,
        # & states for hand 2)
        
        # splitStates is a special attribute to keep track of important state/action pairs during
        # a split. The first position will hold the state/action pair when a split is chosen.
        # The second and third positions will hold all state/action pairs to be rewarded/punished
        # in the first and second hands, respectively.
        self.splitStates = [0,[],[]] 
        
        # Split potential is used to keep track of if the dealt cards makes it possible to split
        self.split_potential = 0
        
        # Split outcome is used to tally up the outcomes from both hands (to reward/punish original
        # decision to split)
        self.splitOutcome = 0
        
        # Split final values is used to store the final player value of each hand in a split
        self.splitFinalValues = [0,0]
        
        # Added attribute to keep track of how well the model is learning
        self.gameTrack = []
        
        # Dict with faces
        self.f_dict = {'A': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6,
                 '7': 7, '8': 8, '9': 9, '10': 10, 'J': 10, 'Q': 10,
                 'K': 10}
    
    # Create stack
    def makeStack(self):
        
        # Create empty stack
        test_stack = []
        
        # Define new list with faces
        f_list = ['A','2','3','4','5','6','7','8','9','10','J','Q','K']
        
        # Extend empty stack by 4*num_decks*(list of cards)
        for i in range(self.num_decks):
            for j in range(4):
                test_stack.extend(f_list)
        
        # Shuffle the stack
        random.shuffle(test_stack)
        
        # Set the new stack
        self.testStack = test_stack
        
    # Deal single card
    def giveCard(self):
        
        # When stack falls below 52 cards, make a new stack
        if (len(self.testStack) < 52):
        
            self.makeStack()
            
        # Remove the first card from the stack and set it to card to deal
        cardToDeal = self.testStack.pop(0)
        
        return cardToDeal
    

    def dealerPolicy(self, current_value, usable_ace, is_end):
        
        if current_value > 21:
            
            # If dealer has a usable ace, convert it from an 11 to a 1 (subtract 10)
            # Otherwise, game is over, dealer busts
            if usable_ace:
                current_value -= 10
                usable_ace = False
            else:
                return current_value, usable_ace, True
            
        # Dealer stays on 17 or greater
        # Otherwise, deal a new card
        if current_value >= 17:
            return current_value, usable_ace, True
        else:
            card = self.giveCard()
            card_value = self.f_dict[card]
            
            # If card is an ace, check current_value and decide if we can convert
            # it to 11 or have to keep it as 1
            if card_value == 1:
                if current_value <= 10:
                    return current_value + 11, True, False
                return current_value + 1, usable_ace, False
            else:
                return current_value + card_value, usable_ace, False
    
    # Method for action to be chosen by player.
    # curRound and trainRound are used to keep track of what percentage of games
    # we've iterated through. This is used to adjust the exploration rate during sim.
    
    def chooseAction(self,curRound=0,trainRound=1000):
        
        # Player's current hand value is the first element of self.state
        current_value = self.state[0]

        # Whether to 'explore' or follow policy -- current code gradually shifts towards policy following
        # as more rounds are completed
        
        # Note that even the exploration phase has some simple rules built in, such as never staying
        # below 12, always splitting with two aces, etc. This is to avoid exploring obviously bad
        # strategies.
        
        # ********* Don't forget to switch back!!! **********
        #if (np.random.uniform(0, 1) <= self.exp_rate):
        
            # ********* Shane's Updates: **********
            # ********* STANDARD RL: **********
        #if (np.random.uniform(0, 1) <= self.exp_rate*(1 - curRound/trainRound)):
            # ********* RANDOM (using basic guidance; if 1=1, else statement is ignored): **********
            # *********     (else statement is where RL occurs and the decision is guided by Q Value) **********
        if (1 == 1):
            
            # 1) Obviously if current_value = 21, stay (0)
            # 2) If current_value < 12, exclude staying (0)
            # 3) If this is a second or later action, exclude double downs (2) and splits (3)
            if (current_value == 21):
                action = 0
                
            # If current hand value is greater than 11...
            elif (current_value > 11):
                
                # Allow all actions if it's first action and splits available
                if (self.actionCount == 0) and (self.split_potential == 1):
                    action = np.random.choice(self.actions)
                    
                # Exclude splits (3) if self.split_potential = 0
                elif (self.actionCount == 0):
                    action = np.random.choice(self.actions[0:3])
                    
                # Exclude splits and double downs if it's after first action
                else:
                    action = np.random.choice(self.actions[0:2])
                    
            # If the hand value is 2, then it's two aces --> split
            elif (current_value == 2):
                action = 3
                
            # Else, the hand value is less than 12 but not 2
            # Never stay!
            else:
                
                # If split potential is 1 and it's first action,
                # we can hit, double, split. If it's not first
                # action, we can only hit.
                if (self.split_potential == 1):
                    if (self.actionCount == 0):
                        action = np.random.choice([1,2,3])
                    else:
                        action = 1
                        
                # If it's first action with no split potential,
                # we can hit or double. Otherwise, we can only
                # hit.
                else:
                    if (self.actionCount == 0):
                        action = np.random.choice([1,2])
                    else:
                        action = 1
        
        # If we don't 'explore,' then we're following Q value policy
        else:
            
            # Use our basic rules to set an action if we can
            if current_value == 21:
                action = 0
            elif current_value == 2:
                action = 3
                
            # Otherwise we go through checking our Q scores
            else:
                # Initialize a 'v' variable to compare against first Q value and set a default
                # action of staying
                v = -9999999
                action = 0
                
                # Check each action's Q value for that state -- if it's higher than previous Q value,
                # make this the new chosen action.

                # Note that we skip checking some actions, as they cannot be performed with
                # certain states
                for a in self.player_Q_Values[self.state]:

                    # If we've already made a prior action, we can't double down or split
                    # Therefore, skip these actions in the loop
                    if ((self.actionCount > 0) and (a > 1)):        
                        continue

                    # If there's no split potential, skip splitting as a choice
                    if ((self.split_potential == 0) and (a == 3)):
                        continue

                    # if the above two conditions aren't true, all actions are on the table
                    if self.player_Q_Values[self.state][a] > v:
                        action = a
                        v = self.player_Q_Values[self.state][a]
        
        
        # Increment our action count
        self.actionCount += 1
        
        return action

    # Method to create the next state, based on the chosen action, and check if game
    # will continue
    def playerNxtState(self, action):
        current_value = self.state[0]
        show_card = self.state[1]
        usable_ace = self.state[2]

        # Action is a HIT
        if (action == 1):
            
            card = self.giveCard()
            
            # If dealt card is an ace, check if current value is <= 10.
            # If so, we can use ace as 11 (usable) and we add it to current value.
            # Otherwise, ace is 1, and we add 1 to current value
            if self.f_dict[card] == 1:
                if current_value <= 10:
                    current_value += 11
                    usable_ace = True
                else:
                    current_value += 1
            else:
                current_value += self.f_dict[card]
        
        # Action is a DOUBLE DOWN
        elif (action == 2):

            # We need to keep track of original bet (for state/action pairs)
            # but need to double the bet for rewarding/punishing Q value
            originalBet = self.bet
            self.bet *= 2
            
            # Deal card / ace logic same as for hitting
            card = self.giveCard()
            if self.f_dict[card] == 1:
                if current_value <= 10:
                    current_value += 11
                    usable_ace = True
                else:
                    current_value += 1
            else:
                current_value += self.f_dict[card]
            
            # End the game since double down results in one hit and nothing else
            # Return the state
            
            self.end = True
            
            return (current_value, show_card, usable_ace, originalBet)
             
        # Action is to STAY
        else:
            
            # End the game and return the state
            self.end = True
            return (current_value, show_card, usable_ace, self.bet)

        # If totaling up cards is greater than 21, need to adjust or end game
        if current_value > 21:
            if usable_ace:
                current_value -= 10
                usable_ace = False
            else:
                self.end = True
                return (current_value, show_card, usable_ace, self.bet)
        
        # If we hit and haven't busted, return the state but keep game going
        return (current_value, show_card, usable_ace, self.bet)

    # Method to check winner
    def winner(self, player_value, dealer_value):
        # player 1 | draw 0 | dealer -1
        winner = 0
        if player_value > 21:
            winner = -1
        else:
            if dealer_value > 21:
                winner = 1
            else:
                if player_value < dealer_value:
                    winner = -1
                elif player_value > dealer_value:
                    winner = 1
                else:
                    winner = 0
        return winner

    # Method to update Q values
    def _giveCredit(self, player_value, dealer_value, splitCheck=False, ind_splits=False, split=0):
        
        # We are updating the Q value associated with the decision to split
        if splitCheck:
            
            # Use the outcome count in splitoutcome with the bet to find total reward for split
            # E.g., two wins with an original bet of $10 results in an award of 10*2=20
            # A win and a draw with an original bet of $50 results in an award of 50*1=50
            # A loss and a draw with an original bet of $100 results in an award of 100*-1=-100
            award = self.splitOutcome*self.bet
            
            # We don't need a loop for this state/action pair since there's only one action (when we split)
            s = self.splitStates[0]
            
            # Set each part of our pair to the proper variable and update reward
            state, action = s[0], s[1]
            
            reward = self.player_Q_Values[state][action] + self.lr*(award - self.player_Q_Values[state][action])
            self.player_Q_Values[state][action] = round(reward, 3)
            
        # We are updating the Q value for actions associated with playing each hand of a split
        elif ind_splits:
            
            # Check who won this hand and create reward
            whoWon = self.winner(player_value, dealer_value)
            award = whoWon*self.bet
            
            # Backpropagate the reward through our list of state/action pairs
            # if split=0, we take self.splitStates[1] which is the 2nd slot (used for first hand)
            # if split=1, we take self.splitStates[2] which is the 3rd slot (used for second hand)
            for s in reversed(self.splitStates[split+1]):
                state, action = s[0], s[1]
                
                reward = self.player_Q_Values[state][action] + self.lr*(award - self.player_Q_Values[state][action])
                self.player_Q_Values[state][action] = round(reward, 3)
        
        # We are updating the Q value for non-split-associated actions
        else:
            whoWon = self.winner(player_value, dealer_value)
            award = whoWon*self.bet

            # backpropagate the reward through our list of state/action pairs
            # for non-split-associated moves, these are stored in player_state_action
            for s in reversed(self.player_state_action):
                state, action = s[0], s[1]
                reward = self.player_Q_Values[state][action] + self.lr*(award - self.player_Q_Values[state][action])
                self.player_Q_Values[state][action] = round(reward, 3)
            
            
    # Reset for new game/round
    def reset(self):
        self.player_state_action = []
        self.state = (0, 0, False, 10)  # initial state
        self.end = False
        self.actionCount = 0
        self.bet = 0
        self.split_potential = 0
        self.splitStates = [0,[],[]]
        self.splitOutcome = 0
        self.splitFinalValues = [0,0]

    # Show is set to False if cards go to player, True if cards go to house
    def deal2cards(self, show=False):
        
        # return value, usable_ace, and split_potential after two cards dealt
        # so initialize those here
        value, usable_ace = 0, False
        
        cards = [self.giveCard(), self.giveCard()]
        
        # If cards dealt to player are the same, set split potential to 1
        if (cards[0] == cards[1]) and (show == False):
            self.split_potential = 1
        
        # Create a list of card values from our cards
        card_values = [self.f_dict[cards[0]],self.f_dict[cards[1]]]
        
        # If we have two aces, we'll consider our value as 2 if it's the player
        # Otherwise for the dealer, since the dealer can't split, we'll consider it as 12
        if (card_values[0] == 1) and (card_values[1] == 1):
            
            if show:
                value = 12
                usable_ace = True
            else:
                value = 2
                usable_ace = True
            
        # If we make it to this condition, but it's True, we have one ace
        elif 1 in card_values:
            
            # Sum(card_values) = card + Ace
            # Since Ace is stored as a value of 1, we need to add 10 more to make the Ace 11
            value = sum(card_values) + 10
            usable_ace = True
            
        # Else no aces
        else:
            value = sum(card_values)
            usable_ace = False

        # If dealer, also return the show card
        if show:
            return value, usable_ace, card_values[0]
        else:
            return value, usable_ace

    # Time to define the play method
    def play(self, rounds=1000):
        
        # Iterate through the number of rounds specified
        for i in range(rounds):
            
            # Create bet (just random choice of three values)
            self.bet = random.choice([10,50,100])
            
            # Deal cards to player and dealer
            dealer_value, d_usable_ace, show_card = self.deal2cards(show=True)
            player_value, p_usable_ace = self.deal2cards(show=False)
            
            # Set the current state of the player
            self.state = (player_value, show_card, p_usable_ace, self.bet)
            
            # Check to see if there was blackjack after two cards dealt 
            if player_value == 21 or dealer_value == 21:
                
                whoWon = self.winner(player_value, dealer_value)
            
                if whoWon == 1:
                    self.gameTrack.append(1)
                else:
                    self.gameTrack.append(0)
            
            # Otherwise, let's make some moves!
            else:
                
                # Make a decision for our first action
                firstaction = self.chooseAction(i,rounds)
                
                # If the first action is a split, we need to follow this path
                if (firstaction == 3):
                    
                    # Place initial state/action pair in first slot of splitStates
                    self.splitStates[0] = [self.state, 3]
                    
                    # Since we split, we need half the value of the player_value
                    single_card = int(player_value/2)
                    
                    # Loop through both hands in the split
                    for split in range(0,2):
                        
                        # Set the current value equal to the single card value
                        current_value = single_card
                        
                        # Need to deal a card to the hand
                        card = self.giveCard()
                        
                        # If we draw an ace, it is usable so we add 11 to current value.
                        # Current value is just the single card value so we add the
                        # entirety of the ace value (11) rather than just 10
                        if self.f_dict[card] == 1:
                            current_value += 11
                            usable_ace = True
                            
                        # Otherwise we just add the card value to current value
                        else:
                            current_value += self.f_dict[card]
                        
                        # Need to check for 21 with two cards of the hand
                        # If there is a 21, there are no action decisions
                        # to be made within the individual hand.
                        # Continue to next hand
                        
                        if current_value == 21:
                            
                            self.splitFinalValues[split] = 21
                            continue
                        
                        # Set the state with two cards for the hand
                        self.state = (current_value, show_card, p_usable_ace, self.bet)
                        
                        # Run through hand
                        while True:          
                            
                            # Choose an action for this hand
                            action = self.chooseAction(i,rounds)
                            
                            # Append the state/action pair so we can associate the outcome with it
                            state_action_pair = [self.state, action]
                            self.splitStates[split+1].append(state_action_pair)

                            # update next state
                            self.state = self.playerNxtState(action)

                            # End game if self.end = True
                            if self.end:
                                
                                # Set splitFinalValue as the player value of this final state
                                self.splitFinalValues[split] = self.state[0]
                                
                                break
                                
                    # Dealer draws
                    is_end = False
                    while not is_end:
                        dealer_value, d_usable_ace, is_end = self.dealerPolicy(dealer_value, d_usable_ace, is_end)
                    
                    # Get the state (without the bet since it has no impact on win prob) of the decision to split
                    # Need it for tracking wins/total games in stateActionOutcome
                    initialSplit = self.splitStates[0][0][0:3]
                    
                    # Loop through hands to judge winner and reward/punish
                    for split in range(0,2):
                        
                        # Judge the winner of the hand
                        whoWon = self.winner(self.splitFinalValues[split], dealer_value)
                        
                        # Increment split outcome to keep track of total from split
                        self.splitOutcome += whoWon
                        
                        # If player won the hand
                        if whoWon == 1:
                            
                            # Add a win (1) to gameTrack
                            self.gameTrack.append(1)
                            
                            # Add 1 to the wins [0] and total games [1] for 
                            # the state when the decision to split was made
                            self.stateActionOutcome[initialSplit][3][0] += 1
                            self.stateActionOutcome[initialSplit][3][1] += 1
                            
                            # Now update stateActionOutcome for state/actions within the hand
                            for situation in self.splitStates[split+1]:
                                
                                # Add 1 to win/total columns for these state/action combos
                                # Exclude the bet portion of the state ([0:3]) since the 
                                # bet amount has no impact on win probability
                                
                                # situation[0] is state, situation[1] is action
                                self.stateActionOutcome[situation[0][0:3]][situation[1]][0] += 1
                                self.stateActionOutcome[situation[0][0:3]][situation[1]][1] += 1
                        
                        # Else if player loses or draws
                        else:
                            
                            # Add a 0 to gameTrack
                            self.gameTrack.append(0)
                            
                            # Add 1 only to total games [1] for the state when the
                            # decision to split was made
                            self.stateActionOutcome[initialSplit][3][1] += 1
                            
                            # Now update stateActionOutcome for state/actions within the hand
                            for situation in self.splitStates[split+1]:
                                
                                # Add 1 to the total column for these state/action combos
                                # Exclude the bet portion of the state ([0:3]) since the 
                                # bet amount has no impact on win probability
                                
                                # situation[0] is state, situation[1] is action
                                self.stateActionOutcome[situation[0][0:3]][situation[1]][1] += 1
                                 
                        # reward and update Q value for individual actions within hand
                        self._giveCredit(player_value, dealer_value, splitCheck=False, ind_splits=True, split=split)
                       
                    # reward and update Q value for decision to split
                    # Since we already checked the outcomes of the two hands, we don't
                    # have player/dealer values to pass along so we just put in placeholder 0's
                    self._giveCredit(0,0,splitCheck=True,ind_splits=False)
                    
                # If not a split, we follow this path
                else: 
                    
                    # Due to the way in which our first action is used to check *split or 
                    # another action*, we must use this first action below if we didn't decide on
                    # a split. However, we need a counter to ensure that we make a new action
                    # if it's a second or later run through the while loop below.
                    
                    counter = 0
                    
                    while True:
                        
                        # If it's the first action, set action = firstaction
                        # first action is where we decide split or something else
                        if (counter == 0):
                            action = firstaction
                        # Otherwise, we choose a new action
                        else:
                            action = self.chooseAction(i,rounds)
                        
                        counter += 1
                        
                        # Append the state/action pair so we can associate the outcome with it
                        state_action_pair = [self.state, action]
                        self.player_state_action.append(state_action_pair)

                        # update next state
                        self.state = self.playerNxtState(action)

                        # End game if self.end = True
                        if self.end:
                            break

                    # dealer's turn
                    is_end = False
                    while not is_end:
                        dealer_value, d_usable_ace, is_end = self.dealerPolicy(dealer_value, d_usable_ace, is_end)

                    # Get player value from final state
                    player_value = self.state[0]
                    
                    # Judge winner
                    whoWon = self.winner(player_value, dealer_value)

                    # If player won
                    if whoWon == 1:
                        
                        # Append 1 to gameTrack
                        self.gameTrack.append(1)
                        
                        # Loop through the state/action pairs of consequence to track win prob
                        # and rewarding/punishing
                        for situation in self.player_state_action:
                            
                            # Add 1 to win/total columns for these state/action combos
                            # Exclude the bet portion of the state ([0:3]) since the 
                            # bet amount has no impact on win probability
                            
                            # situation[0] is state, situation[1] is action
                            self.stateActionOutcome[situation[0][0:3]][situation[1]][0] += 1
                            self.stateActionOutcome[situation[0][0:3]][situation[1]][1] += 1
                    else:
                        
                        # Append 0 to gameTrack
                        self.gameTrack.append(0)
                        
                        # Loop through the state/action pairs of consequence to track win prob
                        # and rewarding/punishing
                        for situation in self.player_state_action:

                            # Add 1 to the total column for these state/action combos
                            # Exclude the bet portion of the state ([0:3]) since the 
                            # bet amount has no impact on win probability
                            
                            # situation[0] is state, situation[1] is action
                            self.stateActionOutcome[situation[0][0:3]][situation[1]][1] += 1
                            

                    # Reward and update Q values for decisions within the hand
                    self._giveCredit(player_value, dealer_value, splitCheck=False, ind_splits=False)
  
            self.reset()
    
    # Use this method to save the developed model policy
    def savePolicy(self, file="policy"):
        fw = open(file, 'wb')
        pickle.dump(self.player_Q_Values, fw)
        fw.close()

    # Use this method to load the developed model policy
    def loadPolicy(self, file="policy"):
        fr = open(file, 'rb')
        self.player_Q_Values = pickle.load(fr)
        fr.close()

    # ******************************************************
    # *** This method is to play dealer with the policy  ***
    # *** However, it needs to be updated if it is to be *** 
    # *** used.                                          ***
    # ******************************************************
    def playWithDealer(self, rounds=1000):
        self.reset()
        self.loadPolicy()
        self.exp_rate = 0

        result = np.zeros(3)  # player [win, draw, lose]
        for _ in range(rounds):
            # hit 2 cards each
            # give 2 cards
            dealer_value, d_usable_ace, show_card = self.deal2cards(show=True)
            player_value, p_usable_ace = self.deal2cards(show=False)

            self.state = (player_value, show_card, p_usable_ace, random.choice([10,50,100]))

            # judge winner after 2 cards
            if player_value == 21 or dealer_value == 21:
                if player_value == dealer_value:
                    result[1] += 1
                elif player_value > dealer_value:
                    result[0] += 1
                else:
                    result[2] += 1
            else:
                # player's turn
                while True:
                    action = self.chooseAction()
                    # update next state
                    self.state = self.playerNxtState(action)
                    if self.end:
                        break

                        # dealer's turn
                is_end = False
                while not is_end:
                    dealer_value, d_usable_ace, is_end = self.dealerPolicy(dealer_value, d_usable_ace, is_end)

                # judge
                player_value = self.state[0]
                # print("player value {} | dealer value {}".format(player_value, dealer_value))
                w = self.winner(player_value, dealer_value)
                if w == 1:
                    result[0] += 1
                elif w == 0:
                    result[1] += 1
                else:
                    result[2] += 1
            self.reset()
        return result

In [ ]:
# sol = BlackJackSolution(lr=0.05,exp_rate=1)

In [ ]:
# sol.play(500000)

In [ ]:
# sol.stateActionOutcome

In [ ]:
# sol.player_Q_Values

Plotting to make sure things look normal

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import itertools
# import math

In [ ]:
# sol.gameTrack
# countWindow = 1000
# sumGames = np.add.reduceat(sol.gameTrack, np.arange(0, len(sol.gameTrack), countWindow))
# # We have to subset the sum games because it will extend a little past where we need it (due to split hands)
# sumGames_sub = sumGames[0:501]

In [ ]:
# # Set up coefficients for regression line
# coef = np.polyfit(np.arange(len(sumGames_sub)),sumGames_sub,1)
# poly1d_fn = np.poly1d(coef)

In [ ]:
# f, axes = plt.subplots(1, 1, figsize=(9, 9))

# sns.set_style("darkgrid")
# sns.lineplot(data=sumGames_sub,label=f'Wins per {countWindow}')
# sns.lineplot(data=poly1d_fn(np.arange(len(sumGames_sub))), label=f"Wins per {countWindow} (regression)")

# #plt.plot(sumGames)
# #plt.plot(np.arange(len(sumGames)),poly1d_fn(np.arange(len(sumGames))),'--k')

In [ ]:
# lr_l = [0.1, 0.3, 0.5]
# exp_rate_l = [0.3,0.5,1]
# combos = list(itertools.product(lr_l,exp_rate_l))
# f, axes = plt.subplots(3, 3, figsize=(12, 12))

# for index, combo in enumerate(combos):
#     sol = BlackJackSolution(lr=combo[0],exp_rate=combo[1])
#     print(f'Playing Set {index}...')
#     sol.play(10000)
    
#     countWindow = 100
#     sumGames = np.add.reduceat(sol.gameTrack, np.arange(0, len(sol.gameTrack), countWindow))
    
#     # Create regression 
#     coef = np.polyfit(np.arange(len(sumGames)),sumGames,1)
#     poly1d_fn = np.poly1d(coef)
    
#     sns.lineplot(data=poly1d_fn(np.arange(len(sumGames))), ax=axes[math.floor(index/3),index % 3])
#     sns.lineplot(data=sumGames,ax=axes[math.floor(index/3),index % 3])

In [ ]:
# sol.player_Q_Values

In [3]:
# Import additional dependencies
import pandas as pd
import itertools

In [4]:
# Create list of Learning and Exploration Rate combinations to run RL simulation through
# ***** USE FOR RL (Learning & Exploration Rate) combos *****
learningRateList = [0.1, 0.3, 0.5]
# learningRateList = [0, 0.1, 0.3, 0.5, 1]
explorationRateList = [0.3, 0.5, 1]
# explorationRateList = [0, 0.1, 0.3, 0.5, 1]

# ***** USE FOR RANDOM (no LR & ER combos) *****
random_learningRateList = [1]
random_explorationRateList = [1]

# Create comboLearningAndExplorationRateList by making list of all permutations
comboLearningAndExplorationRateList = list(itertools.product(learningRateList,explorationRateList))
random_comboLearningAndExplorationRateList = list(itertools.product(random_learningRateList,random_explorationRateList))

# Visualize the comboLearningAndExplorationRateList
print(comboLearningAndExplorationRateList)
print(random_comboLearningAndExplorationRateList)

[(0.1, 0.3), (0.1, 0.5), (0.1, 1), (0.3, 0.3), (0.3, 0.5), (0.3, 1), (0.5, 0.3), (0.5, 0.5), (0.5, 1)]
[(1, 1)]


In [5]:
# Define number of games to have RL Simulation run through for each Learning and Exploration Rate Combo
numberOfGamesToPlay = 1000000

# Create lists to hold values for all data (stateActionOutcome, player_Q_Values and gameTrack)
comboLearningAndExplorationRateData = []

# Create lists to hold values for stateActionOutcome data
learningRate = []
explorationRate = []
playerHandValue = []
dealerUpCardValue = []
usableAce = []
standGamesWon = []
standTotalGames = []
standAvgWinRate = []
hitGamesWon = []
hitTotalGames = []
hitAvgWinRate = []
doubleDownGamesWon = []
doubleDownTotalGames = []
doubleDownAvgWinRate = []
splitGamesWon = []
splitTotalGames = []
splitAvgWinRate = []
joinKey = []

# Create lists to hold values for player_Q_Values data
learningRate_Q = []
explorationRate_Q = []
playerHandValue_Q = []
dealerUpCardValue_Q = []
usableAce_Q = []
bet_Q = []
standQvalue = []
hitQvalue = []
doubleDownQvalue = []
splitQvalue = []
joinKey_Q = []

# Create lists to hold values for gameTrack data
learningRate_gt = []
explorationRate_gt = []
joinKey_gt = []
gameTrackData = []

# **********************
# ***** Use for RL *****
# **********************
# Loop through Combinations of learningRate & explorationRate
for index, combo in enumerate(comboLearningAndExplorationRateList):
    print(f'Playing set {index + 1} (lr: {combo[0]}; exp_rate: {combo[1]})...')
    sol = BlackJackSolution(lr = combo[0], exp_rate = combo[1])
    sol.play(numberOfGamesToPlay)
    stateActionOutcome = sol.stateActionOutcome
    playerQvalues = sol.player_Q_Values
    gameTrack = sol.gameTrack
    gameSet = {"comboDesc": f"lr: {combo[0]}, er: {combo[1]}",
               "combo": f"{combo[0]}_{combo[1]}",
               "gameTrack": gameTrack,
               "gamesPlayed": len(gameTrack),
               "gamesWon": sum(gameTrack),
               "percentWon": sum(gameTrack) / len(gameTrack),
               "stateActionOutcome": stateActionOutcome,
               "playerQvalues": playerQvalues
              }
    comboLearningAndExplorationRateData.append(gameSet)
    
    # Append value to each list for each item within the stateActionOutcome dictionary
    for state in stateActionOutcome:
        learningRate.append(combo[0])
        explorationRate.append(combo[1])
        joinKey.append(f"{combo[0]}_{combo[1]}_{state[0]}_{state[1]}_{state[2]}")
        
        playerHandValue.append(state[0])
        dealerUpCardValue.append(state[1])
        usableAce.append(state[2])

        actions = sol.stateActionOutcome[state]

        """STAND"""
        standGamesWon.append(actions[0][0])
        standTotalGames.append(actions[0][1])

        if actions[0][1] != 0:
            standAvgWinRate.append(actions[0][0] / actions[0][1])
        else:
            standAvgWinRate.append(0)

        """HIT"""
        hitGamesWon.append(actions[1][0])
        hitTotalGames.append(actions[1][1])

        if actions[1][1] != 0:
            hitAvgWinRate.append(actions[1][0] / actions[1][1])
        else:
            hitAvgWinRate.append(0)

        """DOUBLE DOWN"""    
        doubleDownGamesWon.append(actions[2][0])
        doubleDownTotalGames.append(actions[2][1])

        if actions[2][1] != 0:
            doubleDownAvgWinRate.append(actions[2][0] / actions[2][1])
        else:
            doubleDownAvgWinRate.append(0)

        """SPLIT"""    
        splitGamesWon.append(actions[3][0])
        splitTotalGames.append(actions[3][1])

        if actions[3][1] != 0:
            splitAvgWinRate.append(actions[3][0] / actions[3][1])
        else:
            splitAvgWinRate.append(0)
            
    # Append value to each list for each item within the player_Q_Values dictionary
    for q in playerQvalues:
        learningRate_Q.append(combo[0])
        explorationRate_Q.append(combo[1])
        joinKey_Q.append(f"{combo[0]}_{combo[1]}_{q[0]}_{q[1]}_{q[2]}")
        
        playerHandValue_Q.append(q[0])
        dealerUpCardValue_Q.append(q[1])
        usableAce_Q.append(q[2])
        bet_Q.append(q[3])

        actionQvalue = sol.player_Q_Values[q]

        """STAND"""
        standQvalue.append(actionQvalue[0])

        """HIT"""
        hitQvalue.append(actionQvalue[1])

        """DOUBLE DOWN"""    
        doubleDownQvalue.append(actionQvalue[2])

        """SPLIT"""    
        splitQvalue.append(actionQvalue[3])



Playing set 1 (lr: 0.1; exp_rate: 0.3)...
Playing set 2 (lr: 0.1; exp_rate: 0.5)...
Playing set 3 (lr: 0.1; exp_rate: 1)...
Playing set 4 (lr: 0.3; exp_rate: 0.3)...
Playing set 5 (lr: 0.3; exp_rate: 0.5)...
Playing set 6 (lr: 0.3; exp_rate: 1)...
Playing set 7 (lr: 0.5; exp_rate: 0.3)...
Playing set 8 (lr: 0.5; exp_rate: 0.5)...
Playing set 9 (lr: 0.5; exp_rate: 1)...


In [7]:
# # **************************************************
# # **************************************************
# # **************************************************
# # **************************************************
# #IMPORTANT: Rerun BlackJackSolution Class cell with updates to chooseAction method:
# #             commenting out RL and uncommenting out RANDOM lines (lines 175 & 178)
# # **************************************************
# # **************************************************
# # **************************************************
# # **************************************************

In [8]:
# **************************
# ***** Use for RANDOM ***** # ***** AFTER ADJUSTING BlackJackSolution class: chooseAction method *****
# **************************
# Loop through RANDOM Combinations of learningRate & explorationRate (only one basic combo; not passed)
for index, combo in enumerate(random_comboLearningAndExplorationRateList):
    print(f'Playing set {index + 1} (random)...')
    sol = BlackJackSolution(lr = combo[0], exp_rate = combo[1])
    sol.play(numberOfGamesToPlay)
    stateActionOutcome = sol.stateActionOutcome
    playerQvalues = sol.player_Q_Values
    gameTrack = sol.gameTrack
    gameSet = {"comboDesc": "random",
               "combo": "random",
               "gameTrack": gameTrack,
               "gamesPlayed": len(gameTrack),
               "gamesWon": sum(gameTrack),
               "percentWon": sum(gameTrack) / len(gameTrack),
               "stateActionOutcome": stateActionOutcome,
               "playerQvalues": playerQvalues
              }
    comboLearningAndExplorationRateData.append(gameSet)
    
    # Append value to each list for each item within the stateActionOutcome dictionary
    for state in stateActionOutcome:
        learningRate.append(combo[0])
        explorationRate.append(combo[1])
        joinKey.append(f"random_{state[0]}_{state[1]}_{state[2]}")
        
        playerHandValue.append(state[0])
        dealerUpCardValue.append(state[1])
        usableAce.append(state[2])

        actions = sol.stateActionOutcome[state]

        """STAND"""
        standGamesWon.append(actions[0][0])
        standTotalGames.append(actions[0][1])

        if actions[0][1] != 0:
            standAvgWinRate.append(actions[0][0] / actions[0][1])
        else:
            standAvgWinRate.append(0)

        """HIT"""
        hitGamesWon.append(actions[1][0])
        hitTotalGames.append(actions[1][1])

        if actions[1][1] != 0:
            hitAvgWinRate.append(actions[1][0] / actions[1][1])
        else:
            hitAvgWinRate.append(0)

        """DOUBLE DOWN"""    
        doubleDownGamesWon.append(actions[2][0])
        doubleDownTotalGames.append(actions[2][1])

        if actions[2][1] != 0:
            doubleDownAvgWinRate.append(actions[2][0] / actions[2][1])
        else:
            doubleDownAvgWinRate.append(0)

        """SPLIT"""    
        splitGamesWon.append(actions[3][0])
        splitTotalGames.append(actions[3][1])

        if actions[3][1] != 0:
            splitAvgWinRate.append(actions[3][0] / actions[3][1])
        else:
            splitAvgWinRate.append(0)
            
    # Append value to each list for each item within the player_Q_Values dictionary
    for q in playerQvalues:
        learningRate_Q.append(combo[0])
        explorationRate_Q.append(combo[1])
        joinKey_Q.append(f"random_{q[0]}_{q[1]}_{q[2]}")
        
        playerHandValue_Q.append(q[0])
        dealerUpCardValue_Q.append(q[1])
        usableAce_Q.append(q[2])
        bet_Q.append(q[3])

        actionQvalue = sol.player_Q_Values[q]

        """STAND"""
        standQvalue.append(actionQvalue[0])

        """HIT"""
        hitQvalue.append(actionQvalue[1])

        """DOUBLE DOWN"""    
        doubleDownQvalue.append(actionQvalue[2])

        """SPLIT"""    
        splitQvalue.append(actionQvalue[3])     

Playing set 1 (random)...


In [9]:
# Create stateActionOutcome_df DataFrame from each of the lists by zipping them together
stateActionOutcome_df = pd.DataFrame(list(zip(learningRate,\
                                              explorationRate,\
                                              playerHandValue,\
                                              dealerUpCardValue,\
                                              usableAce,\
                                              standGamesWon,\
                                              standTotalGames,\
                                              standAvgWinRate,\
                                              hitGamesWon,\
                                              hitTotalGames,\
                                              hitAvgWinRate,\
                                              doubleDownGamesWon,\
                                              doubleDownTotalGames,\
                                              doubleDownAvgWinRate,\
                                              splitGamesWon,\
                                              splitTotalGames,\
                                              splitAvgWinRate,\
                                              joinKey
                                             )))

# Rename stateActionOutcome_df columns
stateActionOutcome_df = stateActionOutcome_df.rename(columns = {0: "learningRate",\
                                                                1: "explorationRate",\
                                                                2: "playerHandValue",
                                                                3: "dealerUpCardValue",
                                                                4: "usableAce",
                                                                5: "standGamesWon",
                                                                6: "standTotalGames",
                                                                7: "standAvgWinRate",
                                                                8: "hitGamesWon",
                                                                9: "hitTotalGames",
                                                                10: "hitAvgWinRate",
                                                                11: "doubleDownGamesWon",
                                                                12: "doubleDownTotalGames",
                                                                13: "doubleDownAvgWinRate",
                                                                14: "splitGamesWon",
                                                                15: "splitTotalGames",
                                                                16: "splitAvgWinRate",
                                                                17: "joinKey"
                                                               })

# Create player_Q_Values_df DataFrame from each of the lists by zipping them together
player_Q_Values_df = pd.DataFrame(list(zip(learningRate_Q,\
                                           explorationRate_Q,\
                                           playerHandValue_Q,\
                                           dealerUpCardValue_Q,\
                                           usableAce_Q,\
                                           bet_Q,\
                                           standQvalue,\
                                           hitQvalue,\
                                           doubleDownQvalue,\
                                           splitQvalue,\
                                           joinKey_Q
                                          )))

# Rename player_Q_Values_df columns
player_Q_Values_df = player_Q_Values_df.rename(columns = {0: "learningRate_Q",\
                                                          1: "explorationRate_Q",\
                                                          2: "playerHandValue_Q",
                                                          3: "dealerUpCardValue_Q",
                                                          4: "usableAce_Q",
                                                          5: "bet_Q",
                                                          6: "standQvalue",
                                                          7: "hitQvalue",
                                                          8: "doubleDownQvalue",
                                                          9: "splitQvalue",
                                                          10: "joinKey_Q"
                                                         })


In [10]:
# Visualize stateActionOutcome_df
stateActionOutcome_df

,learningRate,explorationRate,playerHandValue,dealerUpCardValue,usableAce,standGamesWon,standTotalGames,standAvgWinRate,hitGamesWon,hitTotalGames,hitAvgWinRate,doubleDownGamesWon,doubleDownTotalGames,doubleDownAvgWinRate,splitGamesWon,splitTotalGames,splitAvgWinRate,joinKey
0,0.1,0.3,2,1,True,0,0,0.000000,0,0,0.0,0,0,0.0,206,576,0.357639,0.1_0.3_2_1_True
1,0.1,0.3,2,1,False,0,0,0.000000,0,0,0.0,0,0,0.0,0,0,0.000000,0.1_0.3_2_1_False
2,0.1,0.3,2,2,True,0,0,0.000000,0,0,0.0,0,0,0.0,398,816,0.487745,0.1_0.3_2_2_True
3,0.1,0.3,2,2,False,0,0,0.000000,0,0,0.0,0,0,0.0,0,0,0.000000,0.1_0.3_2_2_False
4,0.1,0.3,2,3,True,0,0,0.000000,0,0,0.0,0,0,0.0,410,804,0.509950,0.1_0.3_2_3_True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1.0,1.0,21,8,False,2408,2583,0.932249,0,0,0.0,0,0,0.0,0,0,0.000000,random_21_8_False
3996,1.0,1.0,21,9,True,419,443,0.945824,0,0,0.0,0,0,0.0,0,0,0.000000,random_21_9_True
3997,1.0,1.0,21,9,False,2446,2590,0.944402,0,0,0.0,0,0,0.0,0,0,0.000000,random_21_9_False
3998,1.0,1.0,21,10,True,1685,1744,0.966170,0,0,0.0,0,0,0.0,0,0,0.000000,random_21_10_True


In [11]:
# Visualize player_Q_Values_df
player_Q_Values_df

,learningRate_Q,explorationRate_Q,playerHandValue_Q,dealerUpCardValue_Q,usableAce_Q,bet_Q,standQvalue,hitQvalue,doubleDownQvalue,splitQvalue,joinKey_Q
0,0.1,0.3,2,1,True,10,-99999999.0,0.0,0.0,8.459101e+03,0.1_0.3_2_1_True
1,0.1,0.3,2,1,True,50,-99999999.0,0.0,0.0,2.649148e+03,0.1_0.3_2_1_True
2,0.1,0.3,2,1,True,100,-99999999.0,0.0,0.0,2.914868e+03,0.1_0.3_2_1_True
3,0.1,0.3,2,1,False,10,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_1_False
4,0.1,0.3,2,1,False,50,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_1_False
...,...,...,...,...,...,...,...,...,...,...,...
11995,1.0,1.0,21,10,True,50,50.0,0.0,0.0,0.000000e+00,random_21_10_True
11996,1.0,1.0,21,10,True,100,100.0,0.0,0.0,0.000000e+00,random_21_10_True
11997,1.0,1.0,21,10,False,10,10.0,0.0,0.0,0.000000e+00,random_21_10_False
11998,1.0,1.0,21,10,False,50,50.0,0.0,0.0,0.000000e+00,random_21_10_False


In [12]:
# Break up player_Q_Values_df into three DataFrames based on the bet value (min = 10; mid = 50; max = 100)
# This is to extend the DataFrame as wide with additional columns for each bet
minBetQvalues_df = player_Q_Values_df.loc[player_Q_Values_df["bet_Q"] == 10]
midBetQvalues_df = player_Q_Values_df.loc[player_Q_Values_df["bet_Q"] == 50]
maxBetQvalues_df = player_Q_Values_df.loc[player_Q_Values_df["bet_Q"] == 100]

In [13]:
# Rename minBetQvalues_df columns 
minBetQvalues_df = minBetQvalues_df.rename(columns={"learningRate_Q": "minBet_learningRate_Q",
                                                    "explorationRate_Q": "minBet_explorationRate_Q",
                                                    "playerHandValue_Q": "minBet_playerHandValue",
                                                    "dealerUpCardValue_Q": "minBet_dealerUpCardValue",
                                                    "usableAce_Q": "minBet_usableAce",
                                                    "bet_Q": "minBet_bet",
                                                    "standQvalue": "minBet_standQvalue",
                                                    "hitQvalue": "minBet_hitQvalue",
                                                    "doubleDownQvalue": "minBet_doubleDownQvalue",
                                                    "splitQvalue": "minBet_splitQvalue"
                                                   })

# Rename midBetQvalues_df columns 
midBetQvalues_df = midBetQvalues_df.rename(columns={"learningRate_Q": "midBet_learningRate_Q",
                                                    "explorationRate_Q": "midBet_explorationRate_Q",
                                                    "playerHandValue_Q": "midBet_playerHandValue",
                                                    "dealerUpCardValue_Q": "midBet_dealerUpCardValue",
                                                    "usableAce_Q": "midBet_usableAce",
                                                    "bet_Q": "midBet_bet",
                                                    "standQvalue": "midBet_standQvalue",
                                                    "hitQvalue": "midBet_hitQvalue",
                                                    "doubleDownQvalue": "midBet_doubleDownQvalue",
                                                    "splitQvalue": "midBet_splitQvalue"
                                                   })

# Rename maxBetQvalues_df columns 
maxBetQvalues_df = maxBetQvalues_df.rename(columns={"learningRate_Q": "maxBet_learningRate_Q",
                                                    "explorationRate_Q": "maxBet_explorationRate_Q",
                                                    "playerHandValue_Q": "maxBet_playerHandValue",
                                                    "dealerUpCardValue_Q": "maxBet_dealerUpCardValue",
                                                    "usableAce_Q": "maxBet_usableAce",
                                                    "bet_Q": "maxBet_bet",
                                                    "standQvalue": "maxBet_standQvalue",
                                                    "hitQvalue": "maxBet_hitQvalue",
                                                    "doubleDownQvalue": "maxBet_doubleDownQvalue",
                                                    "splitQvalue": "maxBet_splitQvalue"
                                                   })

In [14]:
# Visualize minBetQvalues_df
minBetQvalues_df

,minBet_learningRate_Q,minBet_explorationRate_Q,minBet_playerHandValue,minBet_dealerUpCardValue,minBet_usableAce,minBet_bet,minBet_standQvalue,minBet_hitQvalue,minBet_doubleDownQvalue,minBet_splitQvalue,joinKey_Q
0,0.1,0.3,2,1,True,10,-99999999.0,0.0,0.0,8.459101e+03,0.1_0.3_2_1_True
3,0.1,0.3,2,1,False,10,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_1_False
6,0.1,0.3,2,2,True,10,-99999999.0,0.0,0.0,6.473000e+01,0.1_0.3_2_2_True
9,0.1,0.3,2,2,False,10,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_2_False
12,0.1,0.3,2,3,True,10,-99999999.0,0.0,0.0,1.610660e+02,0.1_0.3_2_3_True
...,...,...,...,...,...,...,...,...,...,...,...
11985,1.0,1.0,21,8,False,10,10.0,0.0,0.0,0.000000e+00,random_21_8_False
11988,1.0,1.0,21,9,True,10,-10.0,0.0,0.0,0.000000e+00,random_21_9_True
11991,1.0,1.0,21,9,False,10,10.0,0.0,0.0,0.000000e+00,random_21_9_False
11994,1.0,1.0,21,10,True,10,-10.0,0.0,0.0,0.000000e+00,random_21_10_True


In [15]:
# Visualize midBetQvalues_df
midBetQvalues_df

,midBet_learningRate_Q,midBet_explorationRate_Q,midBet_playerHandValue,midBet_dealerUpCardValue,midBet_usableAce,midBet_bet,midBet_standQvalue,midBet_hitQvalue,midBet_doubleDownQvalue,midBet_splitQvalue,joinKey_Q
1,0.1,0.3,2,1,True,50,-99999999.0,0.0,0.0,2.649148e+03,0.1_0.3_2_1_True
4,0.1,0.3,2,1,False,50,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_1_False
7,0.1,0.3,2,2,True,50,-99999999.0,0.0,0.0,7.267700e+01,0.1_0.3_2_2_True
10,0.1,0.3,2,2,False,50,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_2_False
13,0.1,0.3,2,3,True,50,-99999999.0,0.0,0.0,4.178400e+01,0.1_0.3_2_3_True
...,...,...,...,...,...,...,...,...,...,...,...
11986,1.0,1.0,21,8,False,50,50.0,0.0,0.0,0.000000e+00,random_21_8_False
11989,1.0,1.0,21,9,True,50,50.0,0.0,0.0,0.000000e+00,random_21_9_True
11992,1.0,1.0,21,9,False,50,-50.0,0.0,0.0,0.000000e+00,random_21_9_False
11995,1.0,1.0,21,10,True,50,50.0,0.0,0.0,0.000000e+00,random_21_10_True


In [16]:
# Visualize maxBetQvalues_df
maxBetQvalues_df

,maxBet_learningRate_Q,maxBet_explorationRate_Q,maxBet_playerHandValue,maxBet_dealerUpCardValue,maxBet_usableAce,maxBet_bet,maxBet_standQvalue,maxBet_hitQvalue,maxBet_doubleDownQvalue,maxBet_splitQvalue,joinKey_Q
2,0.1,0.3,2,1,True,100,-99999999.0,0.0,0.0,2.914868e+03,0.1_0.3_2_1_True
5,0.1,0.3,2,1,False,100,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_1_False
8,0.1,0.3,2,2,True,100,-99999999.0,0.0,0.0,6.026800e+01,0.1_0.3_2_2_True
11,0.1,0.3,2,2,False,100,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_2_False
14,0.1,0.3,2,3,True,100,-99999999.0,0.0,0.0,1.530750e+02,0.1_0.3_2_3_True
...,...,...,...,...,...,...,...,...,...,...,...
11987,1.0,1.0,21,8,False,100,100.0,0.0,0.0,0.000000e+00,random_21_8_False
11990,1.0,1.0,21,9,True,100,100.0,0.0,0.0,0.000000e+00,random_21_9_True
11993,1.0,1.0,21,9,False,100,100.0,0.0,0.0,0.000000e+00,random_21_9_False
11996,1.0,1.0,21,10,True,100,100.0,0.0,0.0,0.000000e+00,random_21_10_True


In [17]:
# Join/merge minBetQvalues_df and midBetQvalues_df on joinKey_Q to create qValues_df
qValues_df = pd.merge(minBetQvalues_df, midBetQvalues_df, how = "outer",\
                      on = "joinKey_Q", suffixes = ("_minBet","_midBet"))

# Join/merge maxBetQvalues_df to qValues_df
qValues_df = qValues_df.merge(maxBetQvalues_df, how = "outer",\
                      on = "joinKey_Q", suffixes = ("_midBet","_maxBet"))

# # Join/merge minBetQvalues_df and midBetQvalues_df on in-common columns to create qValues_df
# qValues_df = pd.merge(minBetQvalues_df, midBetQvalues_df, how = "outer",\
#                       left_on = ["minBet_learningRate_Q", "minBet_explorationRate_Q", "minBet_playerHandValue", "minBet_dealerUpCardValue", "minBet_usableAce"],\
#                       right_on = ["midBet_learningRate_Q", "midBet_explorationRate_Q", "midBet_playerHandValue", "midBet_dealerUpCardValue", "midBet_usableAce"]
#                      )

# # Join/merge maxBetQvalues_df to qValues_df
# qValues_df = qValues_df.merge(maxBetQvalues_df, how = "outer",\
#                       left_on = ["midBet_learningRate_Q", "midBet_explorationRate_Q", "midBet_playerHandValue", "midBet_dealerUpCardValue", "midBet_usableAce"],\
#                       right_on = ["maxBet_learningRate_Q", "maxBet_explorationRate_Q", "maxBet_playerHandValue", "maxBet_dealerUpCardValue", "maxBet_usableAce"]
#                      )

# Drop unnecessary columns
qValues_df = qValues_df[["minBet_bet", "minBet_standQvalue", "minBet_hitQvalue", "minBet_doubleDownQvalue", "minBet_splitQvalue",\
                         "midBet_bet", "midBet_standQvalue", "midBet_hitQvalue", "midBet_doubleDownQvalue", "midBet_splitQvalue",\
                         "maxBet_bet", "maxBet_standQvalue", "maxBet_hitQvalue", "maxBet_doubleDownQvalue", "maxBet_splitQvalue",\
                         "joinKey_Q"]]

# Visualize qValues_df
qValues_df

,minBet_bet,minBet_standQvalue,minBet_hitQvalue,minBet_doubleDownQvalue,minBet_splitQvalue,midBet_bet,midBet_standQvalue,midBet_hitQvalue,midBet_doubleDownQvalue,midBet_splitQvalue,maxBet_bet,maxBet_standQvalue,maxBet_hitQvalue,maxBet_doubleDownQvalue,maxBet_splitQvalue,joinKey_Q
0,10,-99999999.0,0.0,0.0,8.459101e+03,50,-99999999.0,0.0,0.0,2.649148e+03,100,-99999999.0,0.0,0.0,2.914868e+03,0.1_0.3_2_1_True
1,10,-99999999.0,0.0,0.0,1.000000e+08,50,-99999999.0,0.0,0.0,1.000000e+08,100,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_1_False
2,10,-99999999.0,0.0,0.0,6.473000e+01,50,-99999999.0,0.0,0.0,7.267700e+01,100,-99999999.0,0.0,0.0,6.026800e+01,0.1_0.3_2_2_True
3,10,-99999999.0,0.0,0.0,1.000000e+08,50,-99999999.0,0.0,0.0,1.000000e+08,100,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_2_False
4,10,-99999999.0,0.0,0.0,1.610660e+02,50,-99999999.0,0.0,0.0,4.178400e+01,100,-99999999.0,0.0,0.0,1.530750e+02,0.1_0.3_2_3_True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,10,10.0,0.0,0.0,0.000000e+00,50,50.0,0.0,0.0,0.000000e+00,100,100.0,0.0,0.0,0.000000e+00,random_21_8_False
3996,10,-10.0,0.0,0.0,0.000000e+00,50,50.0,0.0,0.0,0.000000e+00,100,100.0,0.0,0.0,0.000000e+00,random_21_9_True
3997,10,10.0,0.0,0.0,0.000000e+00,50,-50.0,0.0,0.0,0.000000e+00,100,100.0,0.0,0.0,0.000000e+00,random_21_9_False
3998,10,-10.0,0.0,0.0,0.000000e+00,50,50.0,0.0,0.0,0.000000e+00,100,100.0,0.0,0.0,0.000000e+00,random_21_10_True


In [18]:
# Join/merge stateActionOutcome_df and qValues_df on joinKey = joinKey_Q to create blackJackGameData_df
blackJackGameData_df = pd.merge(stateActionOutcome_df, qValues_df, how = "outer",\
                                left_on = "joinKey", right_on = "joinKey_Q"
                               )

# Visualize blackJackGameData_df
blackJackGameData_df

,learningRate,explorationRate,playerHandValue,dealerUpCardValue,usableAce,standGamesWon,standTotalGames,standAvgWinRate,hitGamesWon,hitTotalGames,...,midBet_standQvalue,midBet_hitQvalue,midBet_doubleDownQvalue,midBet_splitQvalue,maxBet_bet,maxBet_standQvalue,maxBet_hitQvalue,maxBet_doubleDownQvalue,maxBet_splitQvalue,joinKey_Q
0,0.1,0.3,2,1,True,0,0,0.000000,0,0,...,-99999999.0,0.0,0.0,2.649148e+03,100,-99999999.0,0.0,0.0,2.914868e+03,0.1_0.3_2_1_True
1,0.1,0.3,2,1,False,0,0,0.000000,0,0,...,-99999999.0,0.0,0.0,1.000000e+08,100,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_1_False
2,0.1,0.3,2,2,True,0,0,0.000000,0,0,...,-99999999.0,0.0,0.0,7.267700e+01,100,-99999999.0,0.0,0.0,6.026800e+01,0.1_0.3_2_2_True
3,0.1,0.3,2,2,False,0,0,0.000000,0,0,...,-99999999.0,0.0,0.0,1.000000e+08,100,-99999999.0,0.0,0.0,1.000000e+08,0.1_0.3_2_2_False
4,0.1,0.3,2,3,True,0,0,0.000000,0,0,...,-99999999.0,0.0,0.0,4.178400e+01,100,-99999999.0,0.0,0.0,1.530750e+02,0.1_0.3_2_3_True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1.0,1.0,21,8,False,2408,2583,0.932249,0,0,...,50.0,0.0,0.0,0.000000e+00,100,100.0,0.0,0.0,0.000000e+00,random_21_8_False
3996,1.0,1.0,21,9,True,419,443,0.945824,0,0,...,50.0,0.0,0.0,0.000000e+00,100,100.0,0.0,0.0,0.000000e+00,random_21_9_True
3997,1.0,1.0,21,9,False,2446,2590,0.944402,0,0,...,-50.0,0.0,0.0,0.000000e+00,100,100.0,0.0,0.0,0.000000e+00,random_21_9_False
3998,1.0,1.0,21,10,True,1685,1744,0.966170,0,0,...,50.0,0.0,0.0,0.000000e+00,100,100.0,0.0,0.0,0.000000e+00,random_21_10_True


In [19]:
# Export to CSV file to use in other locations (i.e., jupyter notebooks, visual studio, tableau, etc)
blackJackGameData_df.to_csv("blackJackGameData.csv", index = False)

In [20]:
# # Export to JSON file to use in other locations (i.e., jupyter notebooks, visual studio, etc)
# blackJackGameData_df.to_json("blackJackGameData.json")

In [21]:
# Create lists to hold values for gameTrack data
handNumberList = []
handResultList = []
handsWonRunningTotalList = []
avgHandsWonRunningTotalList = []
lr_er_list = []
lr_er_descList = []

# Loop through each combo interation within comboLearningAndExplorationRateData
for game in comboLearningAndExplorationRateData:
    handNumber = 0
    winCounter = 0
    
#     # ALL GAMES: Append value to each list for each item within the comboLearningAndExplorationRateData dictionary
#     for hand in game["gameTrack"]:
#         handNumber += 1
#         winCounter += hand
#         handNumberList.append(handNumber)
#         handResultList.append(hand)
#         handsWonRunningTotalList.append(winCounter)
#         avgHandsWonRunningTotalList.append(str(winCounter / handNumber))
#         lr_er_list.append(game["combo"])
#         lr_er_descList.append(game["comboDesc"])

    # Only append every 10,000th game and final game data to the lists
    interval = 10000
    numberOfHands = len(game["gameTrack"])
    intervalsToAppend = list(range(0,numberOfHands,interval))
    intervalsToAppend.append(numberOfHands)
    intervalsToAppend.pop(0)

    # Append value to each list for each item within the comboLearningAndExplorationRateData dictionary
    for hand in game["gameTrack"]:
        handNumber += 1
        winCounter += hand
        if handNumber in intervalsToAppend:
            handNumberList.append(handNumber)
            handResultList.append(hand)
            handsWonRunningTotalList.append(winCounter)
            avgHandsWonRunningTotalList.append(str(winCounter / handNumber))
            lr_er_list.append(game["combo"])
            lr_er_descList.append(game["comboDesc"])


In [22]:
# Create gameTrack_df DataFrame from each of the lists by zipping them together
blackJackGameTrackerData_df = pd.DataFrame(list(zip(lr_er_list,\
                                                    lr_er_descList,\
                                                    handNumberList,\
                                                    handResultList,\
                                                    handsWonRunningTotalList,\
                                                    avgHandsWonRunningTotalList,\
                                                   )))

# Rename gameTrack_df columns
blackJackGameTrackerData_df = blackJackGameTrackerData_df.rename(columns = {0: "lr_er",\
                                                                            1: "lr_er_desc",\
                                                                            2: "handNumber",\
                                                                            3: "handResult",\
                                                                            4: "handsWonRunningTotal",\
                                                                            5: "avgHandsWonRunningTotal",\
                                                                           })

# Visualize blackJackGameTrackerData_df
blackJackGameTrackerData_df

,lr_er,lr_er_desc,handNumber,handResult,handsWonRunningTotal,avgHandsWonRunningTotal
0,0.1_0.3,"lr: 0.1, er: 0.3",10000,1,3810,0.381
1,0.1_0.3,"lr: 0.1, er: 0.3",20000,1,7648,0.3824
2,0.1_0.3,"lr: 0.1, er: 0.3",30000,1,11589,0.3863
3,0.1_0.3,"lr: 0.1, er: 0.3",40000,0,15563,0.389075
4,0.1_0.3,"lr: 0.1, er: 0.3",50000,0,19385,0.3877
...,...,...,...,...,...,...
1016,random,random,990000,0,312944,0.3161050505050505
1017,random,random,1000000,1,316083,0.316083
1018,random,random,1010000,0,319293,0.31613168316831686
1019,random,random,1020000,0,322481,0.3161578431372549


In [23]:
# Export to CSV file to use in other locations (i.e., jupyter notebooks, visual studio, tableau, etc)
blackJackGameTrackerData_df.to_csv("blackJackGameTrackerData.csv", index = False)

In [24]:
# # Export to JSON file to use in other locations (i.e., jupyter notebooks, visual studio, etc)
# blackJackGameTrackerData_df.to_json("blackJackGameTrackerData.json")

In [31]:
sum(standGamesWon)
sum(standTotalGames)

5939818

In [32]:
len(comboLearningAndExplorationRateData)

10

In [34]:
# for i in comboLearningAndExplorationRateData:
#     print(sum(i["gameTrack"]))

413988
404188
377336
409077
397708
373448
405783
395963
370921
323689


In [55]:
for i in comboLearningAndExplorationRateData:
    testCounter = 0
    for j in i["gameTrack"]:
        testCounter += 1
    print(f'Games Won: {sum(i["gameTrack"])} | Total Games: {testCounter}')

Games Won: 413988 | Total Games: 1014730
Games Won: 404188 | Total Games: 1016078
Games Won: 377336 | Total Games: 1018122
Games Won: 409077 | Total Games: 1013276
Games Won: 397708 | Total Games: 1014832
Games Won: 373448 | Total Games: 1017903
Games Won: 405783 | Total Games: 1012360
Games Won: 395963 | Total Games: 1014510
Games Won: 370921 | Total Games: 1018514
Games Won: 323689 | Total Games: 1023713
